# Experiment on NTCIR-17 Transfer Task Eval Dataset

This notebook shows how to apply BM25 to the eval dataset of NTCIR-17 Transfer Task using [PyTerrier](https://pyterrier.readthedocs.io/en/latest/) (v0.9.2).

## Previous Step

- `preprocess-transfer1-eval-ipynb`

## Requirement

- Java v11

## Path

In [1]:
import os
os.environ['INDEX'] = '../indexes/ntcir17-transfer/eval'
os.environ['RUN'] = '../runs/ntcir17-transfer/eval'

## Datasets

In [2]:
import sys
!{sys.executable} -m pip install -q ir_datasets


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
sys.path.append(os.path.join(os.path.dirname(os.path.abspath('__file__')), '../datasets'))

In [4]:
import ir_datasets
import ntcir_transfer
dataset = ir_datasets.load('ntcir-transfer/1/eval')

## Tokenization

- In this example, we use [SudachiPy](https://github.com/WorksApplications/SudachiPy) (v0.5.4) + sudachidict_core dictionary + SplitMode.A
- Other tokenizers can also be used

In [5]:
import sys
!{sys.executable} -m pip install -q sudachipy sudachidict_core


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import re
import json
from sudachipy import tokenizer
from sudachipy import dictionary
tokenizer_obj = dictionary.Dictionary().create()
mode = tokenizer.Tokenizer.SplitMode.A

In [7]:
def tokenize_text(text):
    atok = ' '.join([m.surface() for m in tokenizer_obj.tokenize(text, mode)])
    return atok

In [8]:
tokenize_text('すもももももももものうち')

'すもも も もも も もも の うち'

## Experiment

### PyTerrier

In [9]:
# Change JAVA_HOME to fit your environment
JAVA_HOME = '/usr/lib/jvm/default'
os.environ['JAVA_HOME'] = JAVA_HOME
os.getenv('JAVA_HOME')

'/usr/lib/jvm/default'

In [10]:
import sys
# !{sys.executable} -m pip install -q python-terrier

In [11]:
import pandas as pd
import pyterrier as pt
if not pt.started():
  pt.init(tqdm='notebook')

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [12]:
dataset_pt = pt.get_dataset('irds:ntcir-transfer/1/eval')

### Indexing

In [13]:
# !rm -rf $INDEX
!mkdir -p $INDEX

In [14]:
indexer = pt.IterDictIndexer(os.getenv('INDEX'))
indexer.setProperty("tokeniser", "UTFTokeniser")
indexer.setProperty("termpipelines", "")

In [15]:
def train_doc_generate():
    for doc in dataset.docs_iter():
        yield { 'docno': doc.doc_id, 'text': tokenize_text(doc.text) }

In [16]:
sys.path.append("../")
import models
from models.jance.jance import PyTDenseIndexer, PyTDenseRetrieval
import importlib
importlib.reload(models.jance.jance)
from models.jance.jance import PyTDenseIndexer, PyTDenseRetrieval

index_path = "../indexes/ntcir17-transfer/jance"

jance_indexer = PyTDenseIndexer(index_path, verbose=False)

/home/kush/Projects/NTCIR17/transfer1/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
%%time
# indexref = indexer.index(train_doc_generate())
index_path = jance_indexer.index(train_doc_generate())

Segment 0


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
!ls $INDEX

data.direct.bf		   data.lexicon.fsomaphash  data.meta.zdata
data.document.fsarrayfile  data.lexicon.fsomapid    data.properties
data.inverted.bf	   data.meta-0.fsomapfile
data.lexicon.fsomapfile    data.meta.idx


### Topics

In [ ]:
def tokenize_topics():
    import re
    code = re.compile('[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％]')
    queries = dataset_pt.get_topics(tokenise_query=False)
    for idx, row in queries.iterrows():
        queries.iloc[idx, 1] = code.sub('', tokenize_text(row.query))
    return queries

In [ ]:
tokenize_topics()

,qid,query
0,0101,Ｂ 型 肝炎
1,0102,異種 膵島 移植
2,0103,高 血圧
3,0104,肺 小 細胞 癌
4,0105,新規 キノロン 剤
5,0106,β ３ アドレナリン 受容 体 遺伝 子 変異
6,0107,塞栓 療法
7,0108,XML
8,0109,TCP の 高速 化
9,0110,情報 検索 の 可視 化


### Retrieval

- The performance value (e.g., nDCG) is expected to be 0.0.
- You can use the generated run files for submission.

In [ ]:
# Load existing index files
indexref = pt.IndexFactory.of(os.getenv('INDEX'))

In [ ]:
!mkdir -p $RUN

In [ ]:
bm25 = pt.BatchRetrieve(indexref, wmodel="BM25")
anceretr = PyTDenseRetrieval(index_path)

In [ ]:
# dummy qrels
import pandas as pd
dummy_qrels = pd.DataFrame(dataset_pt.get_topics(), columns=['qid'])
dummy_qrels['docno'] = 'docno'
dummy_qrels['label'] = 0

In [ ]:
%%time
from pyterrier.measures import *
pt.Experiment(
    [bm25, anceretr],
    tokenize_topics(),
    dummy_qrels,
    eval_metrics=[nDCG],
    names = ["BM25", "JANCE"],
    save_dir = os.getenv('RUN'),
    save_mode = "overwrite"
)

CPU times: user 4.28 s, sys: 2.31 s, total: 6.59 s
Wall time: 2.41 s


,name,nDCG
0,MyRun-BM25,0.0


In [ ]:
!gunzip -c $RUN/MyRun-BM25.res.gz | head

0101 Q0 kaken-j-0911436000 0 21.86485284250732 pyterrier
0101 Q0 kaken-j-0921440800 1 21.733548660790195 pyterrier
0101 Q0 kaken-j-0960142800 2 21.6993557888258 pyterrier
0101 Q0 kaken-j-0975101400 3 21.659867826004042 pyterrier
0101 Q0 kaken-j-0934033100 4 21.651742953172338 pyterrier
0101 Q0 kaken-j-0912100600 5 21.594389793069617 pyterrier
0101 Q0 kaken-j-0882391600 6 21.511995261510908 pyterrier
0101 Q0 kaken-j-0883102100 7 21.46097823369766 pyterrier
0101 Q0 kaken-j-0937129200 8 21.450990698888955 pyterrier
0101 Q0 kaken-j-0941469900 9 21.421462357753665 pyterrier
